# Self Driving Traffic Sign Detection

Isaiah Jenkins

## Import the required libraries

In [1]:
import tensorflow as tf
from keras import layers, Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50

from sklearn.model_selection import train_test_split

import pandas as pd
import os

## 1. About the data

1. a. Description

Throughout this analysis we will explore Udacity's Self Driving dataset. This dataset consists of images for thousands of pedestrians, bikers, cars, and traffic lights. Although traffic light images are underrepresented in the dataset, the focus of the analysis will be based solely on traffic light images.

1. b. Data dictionary, 97,942 labels across 11 classes and 15,000 images, 1,720 null examples (images with no labels).

Class Balance across images (labels in each image)

* car - 64,399 - over represented
* pedestrian - 10,806
* trafficLight-Red - 6,870
* trafficLight-Green - 5,465 - under represented
* truck - 3,623 - under represented
* trafficLight - 2,568 - under represented
* biker - 1,864 - under represented
* trafficLight-RedLeft - 1,751 - under represented
* trafficLight-GreenLeft - 310 - under represented
* trafficLight-Yellow - 272 - under represented
* trafficLight-YellowLeft - 14 - under represented

## 2. Objectives

Throughout this analysis, we will explore and build various deep learning convolutional neural network (CNN) architectures to detect traffic light signs, aiming to optimize accuracy and efficiency. Our objective is to compare different model variations, such as CNNs with different depths, pre-trained models, and data augmentation techniques, to determine the most effective approach. Potential challenges include handling variations in lighting conditions, occlusions, and small object sizes, which may impact detection performance. Additionally, dataset imbalances and misclassifications due to similar-looking traffic signs could introduce biases, requiring careful preprocessing and model tuning.

## 3. Data Exploration, Cleaning and Feature Engineering

* Extract traffic sign & traffic light images from the dataset to help with computational efficiency.
* Resize images to standardize dataset making it computationally efficient making it easier to analyze data.
* Normalize pixel values (0-1 range) to help with generalization.
* Augment data (rotation, brightness shifts, contrast changes) to improve generalization.
* Upsample underrepresented yellow traffic lights to improve model accuracy.

### Load in data

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

!unzip -o -q '/content/gdrive/MyDrive/data.zip' -d '/content/'

Mounted at /content/gdrive


In [3]:
DATASET_PATH = 'data/export'
CSV_PATH = os.path.join(DATASET_PATH, '_annotations.csv')

In [9]:
df = pd.read_csv(CSV_PATH) # load annotations

In [10]:
df = df[df['class'].isin(['trafficLight-Red', 'trafficLight-Yellow', 'trafficLight-Green'])]

In [11]:
# 🔹 Count Class Distribution
print("Original Class Distribution:\n", df["class"].value_counts())

Original Class Distribution:
 class
trafficLight-Red       13673
trafficLight-Green     10838
trafficLight-Yellow      541
Name: count, dtype: int64


In [13]:
# Separate Classes
df_red = df[df["class"] == "trafficLight-Red"]
df_yellow = df[df["class"] == "trafficLight-Yellow"]
df_green = df[df["class"] == "trafficLight-Green"]

In [14]:
# Upsample Yellow traffic light images
max_class_size = max(len(df_red), len(df_green))  # Find largest class count
df_yellow_upsampled = df_yellow.sample(n=max_class_size, replace=True, random_state=42)

In [15]:
# 🔹 Combine the Balanced Dataset
df_balanced = pd.concat([df_red, df_yellow_upsampled, df_green], ignore_index=True)
# 🔹 Shuffle Dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [16]:
# 🔹 Count Class Balanced Distribution
print("Balanced Class Distribution:\n", df_balanced["class"].value_counts())

Balanced Class Distribution:
 class
trafficLight-Red       13673
trafficLight-Yellow    13673
trafficLight-Green     10838
Name: count, dtype: int64


In [71]:
file_names = df_balanced['filename'].values # image files names
labels = df_balanced['class'].values # classes

In [98]:
print('file_names', file_names.shape, 'labels', labels.shape)

file_names (38184,) labels (38184, 3)


In [72]:
label_map = {"trafficLight-Red": 0, "trafficLight-Yellow": 1, "trafficLight-Green": 2, }  # Adjust based on dataset
labels = [label_map[label] for label in labels if label in label_map]  # Convert text labels to integers
labels = to_categorical(labels, num_classes=3)  # Convert to one-hot encoding

In [73]:
#Split Data into Train (80%) and Test (20%)
train_files, test_files, train_labels, test_labels = train_test_split(
    file_names, labels, test_size=0.2, random_state=42, stratify=labels, shuffle=True
)

In [74]:
IMG_SIZE = (224, 224)

In [75]:
# Function to Load & Preprocess Images
def load_and_preprocess_image(file_name, label):
    img_path = os.path.join(DATASET_PATH, file_name)  # Update image folder path
    img = load_img(img_path, target_size=IMG_SIZE)  # Load & Resize Image
    img = img_to_array(img) / 255.0  # Convert to NumPy array & Normalize (0-1)
    return img, label

In [76]:
# Create Generators for Train & Test Data
def data_generator(file_list, label_list):
    for f, l in zip(file_list, label_list):
        yield load_and_preprocess_image(f, l)  # Load one image at a time

In [77]:
BATCH_SIZE = 32

In [63]:
# Create a tf.data.Dataset that Loads Images on Demand
train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(train_files, train_labels),
    output_signature=(
        tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),  # Image Shape
        tf.TensorSpec(shape=(3,), dtype=tf.float32)  # One-hot Encoded Label
    )
).shuffle(1000).batch(BATCH_SIZE).take(32).prefetch(tf.data.experimental.AUTOTUNE)

test_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(test_files, test_labels),
    output_signature=(
        tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),  # Image Shape
        tf.TensorSpec(shape=(3,), dtype=tf.float32)  # One-hot Encoded Label
    )
).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [80]:
len(test_files)

7637

## 4. CNN Models

#### ResNet-50 - Variation 1

In [64]:
# Layers
num_classes = 3

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)


In [65]:
# Compile
model = Model(inputs=base_model.input, outputs=outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 25,688,963 (98.00 MB)

 Trainable params: 25,635,843 (97.79 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [81]:
# Fit on training set and compare against validation set after each epoch
model.fit(train_dataset, epochs=5, validation_data=test_dataset)

Epoch 1/5
     32/Unknown 11s 280ms/step - accuracy: 0.9494 - loss: 0.1180

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


32/32 ━━━━━━━━━━━━━━━━━━━━ 31s 903ms/step - accuracy: 0.9494 - loss: 0.1187 - val_accuracy: 0.2838 - val_loss: 1.2641
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 898ms/step - accuracy: 0.9490 - loss: 0.1679 - val_accuracy: 0.3581 - val_loss: 1.2924
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 889ms/step - accuracy: 0.9555 - loss: 0.1406 - val_accuracy: 0.3445 - val_loss: 1.1173
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 894ms/step - accuracy: 0.9768 - loss: 0.0834 - val_accuracy: 0.2813 - val_loss: 1.1107
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 893ms/step - accuracy: 0.9702 - loss: 0.0856 - val_accuracy: 0.3581 - val_loss: 1.1138


In [82]:
# Evaluate - Loss, Accuracy
model.evaluate(test_dataset)

239/239 ━━━━━━━━━━━━━━━━━━━━ 19s 81ms/step - accuracy: 0.3565 - loss: 1.1149


[1.113835096359253, 0.35812491178512573]

#### RestNet50 - Variation 2 Hyperparameter tuning (Dropout, RMSprop optimizer)

In [91]:
# Layers
num_classes = 3

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)  # Drop 50% of neurons
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)  # Drop 50% again
outputs = layers.Dense(num_classes, activation='softmax')(x)

In [92]:
# Compile
model = Model(inputs=base_model.input, outputs=outputs)
model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 26,212,227 (99.99 MB)

 Trainable params: 26,159,107 (99.79 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [93]:
# Fit variation
model.fit(train_dataset, epochs=5, validation_data=test_dataset)

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.3712 - loss: 3.2397 - val_accuracy: 0.2838 - val_loss: 4273.8750
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 884ms/step - accuracy: 0.4783 - loss: 1.1038 - val_accuracy: 0.3581 - val_loss: 33.5013
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 873ms/step - accuracy: 0.5092 - loss: 1.0759 - val_accuracy: 0.3181 - val_loss: 1.1628
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 877ms/step - accuracy: 0.6667 - loss: 0.7676 - val_accuracy: 0.3581 - val_loss: 2.5403
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 884ms/step - accuracy: 0.7252 - loss: 0.6429 - val_accuracy: 0.2838 - val_loss: 1.1255


In [86]:
# Evaluate - Loss, Accuracy
model.evaluate(test_dataset)

239/239 ━━━━━━━━━━━━━━━━━━━━ 19s 81ms/step - accuracy: 0.3571 - loss: 2.9629


[2.9884872436523438, 0.355898916721344]

#### ResNet-50 - Variation 3 (Replacing average pooling with max pooling, Replacing optimizer to adam, Dropout)

In [99]:
# Layers
num_classes = 3

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = layers.GlobalMaxPooling2D()(base_model.output) # added Max Pooling
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)  # Drop 50% of neurons
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)  # Drop 50% again
outputs = layers.Dense(num_classes, activation='softmax')(x)

In [100]:
# Compile
model = Model(inputs=base_model.input, outputs=outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # added Adam optimizer
model.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_9[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 26,212,227 (99.99 MB)

 Trainable params: 26,159,107 (99.79 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [101]:
# Fit variation
model.fit(train_dataset, epochs=5, validation_data=test_dataset)

Epoch 1/5
     32/Unknown 67s 281ms/step - accuracy: 0.4058 - loss: 13.6714

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


32/32 ━━━━━━━━━━━━━━━━━━━━ 91s 1s/step - accuracy: 0.4058 - loss: 13.5404 - val_accuracy: 0.3581 - val_loss: 120669.5938
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 897ms/step - accuracy: 0.4347 - loss: 3.9549 - val_accuracy: 0.3581 - val_loss: 37.1215
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 893ms/step - accuracy: 0.4666 - loss: 2.4748 - val_accuracy: 0.3581 - val_loss: 9.4857
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 894ms/step - accuracy: 0.5162 - loss: 1.9699 - val_accuracy: 0.3581 - val_loss: 1.1028
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 892ms/step - accuracy: 0.5554 - loss: 1.5300 - val_accuracy: 0.3581 - val_loss: 1.1012


In [90]:
# Evaluate - Loss, Accuracy
model.evaluate(test_dataset)

239/239 ━━━━━━━━━━━━━━━━━━━━ 19s 81ms/step - accuracy: 0.3629 - loss: 1161.7773


[1168.3563232421875, 0.35812491178512573]

### Summary of Models

Throughout this analysis, three variations of the pre-trained model ResNet-50 were trained to utilize the power of transfer learning. In the first variation, the base model of ResNet, dense layers, and the Adam optimizer were used for faster convergence. As a result, this variation exhibited slight overfitting, with a training accuracy of 95% and a validation accuracy of 92%. In the second variation, a dropout layer at a dropout rate of 50% was added. In addition, the optimizer was changed to RMSprop. This variation was also slightly overfitted, achieving a training accuracy of 94% and a validation accuracy of 92%. Lastly, for the third variation, the average pooling layer was replaced with max pooling, the optimizer was reverted to the Adam optimizer and the dropout layer stayed the same. In conclusion, this variation did not overfit, resulting in the best optimization, with a training accuracy of 89% and a validation accuracy of 92%.

## 5. Insights and Key Findings

When training with images, the idea was to figure out how to save time and resources by utilizing the earlier layers of a pre-trained neural network, ResNet-50. After training the variations, two of the variations were slightly overfitting and the last variation did not overfit. The difference between the previous variation and the others was that the pooling layer was changed from average pooling to max pooling. Max pooling allows the model to extract prominent features within an image whereas average pooling focuses on all the features. In this dataset, the focus was based on detecting the traffic light colors specifically. Thus, the last variation of the model was the best optimization for reducing the generalization error.

## 6. Next Steps